![logo.png](logo.png)

## Предобработка

In [1]:
path = 'data/gazeta_train.jsonl'
vector_size = 300

In [2]:
from datetime import datetime


class Article:
    def __init__(self, date, url, summary, title, text):
        self.date = date
        self.url = url
        self.summary = summary
        self.title = title
        self.text = text


def parse_datetime(string):
  return datetime.strptime(string, '%Y-%m-%d %H:%M:%S')


def parse_article_json(obj):
    date = parse_datetime(obj['date'])
    url = obj['url']
    summary = obj['summary']
    title = obj['title']
    text = obj['text']
    return Article(date, url, summary, title, text)

In [3]:
import json


def parse_gazeta(text):
    lines = text.split('\n')
    articles = [parse_article_json(json.loads(line)) for line in lines if line != '']
    return articles


def load_and_parse_gazeta(path):
    f = open(path, 'r')
    text = f.read()
    return parse_gazeta(text)

In [4]:
articles = load_and_parse_gazeta(path)

In [5]:
import nltk
import re
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

In [6]:
ru_stopwords = stopwords.words("russian")
morph = MorphAnalyzer()

In [7]:
def preprocess_text(text):
    new_text = text
    new_text = new_text.lower()  # Привести к нижнему регистру
    new_text = re.sub(r'[^\w\s]', ' ', new_text)  # Убрать пунктуацию
    return new_text

In [8]:
def lemmatize_word(word):
    return morph.parse(word)[0].normal_form


def tokenize_sentence(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in ru_stopwords]  # Убрать стоп слова
    # tokens = [lemmatize_word(token) for token in tokens]
    return tokens


def tokenize_article_body(text):
    sents = nltk.sent_tokenize(text)
    sents = [preprocess_text(sent) for sent in sents]
    sents_words = [tokenize_sentence(sent) for sent in sents]
    return sents_words

In [9]:
len(articles)

60964

In [10]:
sentences = []
for article in articles[:1000]:
    sentences += tokenize_article_body(article.text)

In [11]:
len(sentences)

34113

In [64]:
sentences[0]

['сегодня',
 'транспортный',
 'налог',
 'начисляется',
 'зависимости',
 'мощности',
 'автомобиля',
 'причем',
 'цена',
 'сильных',
 'машин',
 'выше',
 'малолитражек']

## Эмбединг

In [58]:
from gensim.models import FastText, Word2Vec

In [78]:
ft = FastText(sentences=sentences, vector_size=vector_size, min_count=1, window=5, workers=8)

Лемматизация + Word2Vec очень медленнно, FastText тоже не быстрый. А тут одно из двух, поэтому я выбираю третий вариант: предобученная модель

In [124]:
from navec import Navec

path = 'data/navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [148]:
def vectorize(request):
    tokens = tokenize_text(request)
    vector = np.zeros((vector_size))
    num_tokens = len(tokens)
    for word in tokens:
        if word in navec:
            vector += navec[word]
    vector /= num_tokens
    return vector


distance.cosine(vectorize('взятку'), vectorize('Российский чиновник дал взятку')), \
    distance.cosine(vectorize('взятку'), vectorize('Хакеры взломали пентагон')), \
        distance.cosine(vectorize('взятку'), vectorize('взятки'))

(0.37482268287720866, 0.9287834559341686, 0.34254276976931297)

In [79]:
import numpy as np

In [127]:
def text_to_vectors(text):
    sents = tokenize_article_body(text)
    vecs = []
    for sent in sents:
        sent_vec = np.zeros((vector_size))
        num_tokens = 0
        for token in sent:
            if token in navec:
                sent_vec += navec[token]
                num_tokens+=1
        if num_tokens > 0:
            sent_vec /= num_tokens
        # sent_vec /= np.linalg.norm(sent_vec)
        vecs.append(sent_vec)
    return vecs

In [128]:
subsample_size = 1000

In [129]:
for article in articles[:subsample_size]:
    article.summary_vecs = text_to_vectors(article.summary)
    article.title_vecs = text_to_vectors(article.title)
    article.sentences = nltk.sent_tokenize(article.summary)

## Ближайшие соседи и поиск

In [130]:
import annoy

In [131]:
index = annoy.AnnoyIndex(vector_size, 'angular')
counter = 0
for article in articles[:subsample_size]:
    for vec in article.summary_vecs:
        index.add_item(counter, vec)
    for vec in article.title_vecs:
        index.add_item(counter, vec)
    counter+=1

In [132]:
index.build(10, n_jobs=-1)

True

In [143]:
from scipy.spatial import distance


def tokenize_text(text):
    return tokenize_sentence(preprocess_text(text))


def find_nearest_sentence(article, request):
    ind = 0
    min_dist = 1000000
    for i in range(len(article.summary_vecs)):
        dist = distance.cosine(article.summary_vecs[i], request)
        if dist < min_dist:
            min_dist = dist
            ind = i
    return ind


def search(request):
    tokens = tokenize_text(request)
    vector = np.zeros((vector_size))
    num_tokens = len(tokens)
    for word in tokens:
        if word in navec:
            vector += navec[word]
    if num_tokens > 0:
        vector /= num_tokens
        # vector /= np.linalg.norm(vector)
    results = index.get_nns_by_vector(vector, 5,)
    return [(i, articles[i].title, \
        articles[i].sentences[find_nearest_sentence(articles[i], vector)]) \
        for i in results]

In [144]:
search('убийца')

[(273,
  'Убийца не был педофилом',
  'В Екатеринбурге к пожизненному сроку приговорен Сергей Кутнюк, которого за убийства и изнасилования детей милиционеры разыскивали шесть лет.'),
 (390,
  'С убийства Маркелова снят мотив',
  'С предполагаемого убийцы адвоката Станислава Маркелова и журналистки Анастасии Бабуровой сняты обвинения в убийстве антифашиста Александра Рюхина в 2006 году.'),
 (45,
  'Таксист стрелял без остановки',
  'Позже полиция обнаружила в лесу и труп предполагаемого преступника — он застрелился.'),
 (162,
  'Пациент убил врача в упор',
  'В Пущинском научном центре РАН пациент застрелил из ружья заместителя главврача больницы, а затем покончил с собой.'),
 (725,
  'Убийцу опознали по прорезям для глаз',
  'Генерал Сергей Кизюн, сидевший в машине Руслана Ямадаева в момент его убийства, опознал стрелявшего в одном из подсудимых.')]

In [145]:
search('шпионский кодер')

[(819,
  'Шпионский кодер',
  'Доказательств того, что он был связан со шпионской сетью, у следствия так и не появилось.'),
 (777,
  'Шпионский Роман',
  'Арест российских шпионов в конце июня спровоцировал звонок Анны Чепмен в Москву.'),
 (77,
  'Брайант сыграл для Спилберга',
  'На игру в лос-анджелесский «Стейпл Центр» вместе с рядовыми болельщиками пришли немало звезд Голливуда, начиная Стивеном Спилбергом и заканчивая Шарлиз Терон и Крисом Роком.'),
 (80,
  'Фильтруй коммент',
  'Интернет-СМИ должны редактировать комментарии своих читателей.'),
 (81,
  'Фильтруй коммент',
  'Интернет-СМИ должны редактировать комментарии своих читателей.')]